<a href="https://colab.research.google.com/github/chi20227088/OpenAI-Features-and-Functionality---Coding/blob/main/Nh%E1%BA%ADn_di%E1%BB%87n_%E1%BA%A3nh_c%C3%B3_m%E1%BA%B7t_ng%C6%B0%E1%BB%9Di_b%E1%BA%B1ng_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical

# 1. Chuẩn bị dữ liệu
df = pd.read_csv('train_face.csv')
pixels = np.array([list(map(int, row.split())) for row in df['pixels']])
pixels = pixels.reshape(-1, 48, 48, 1) / 255.0  # Chuẩn hóa [0,1]
labels = to_categorical(df['label'], num_classes=2)

x_train, x_test, y_train, y_test = train_test_split(pixels, labels, test_size=0.2, random_state=42)

# 2. Xây dựng mô hình CNN
model = models.Sequential([
    layers.Conv2D(32, (5,5), activation='relu', padding='same', input_shape=(48,48,1)), # Convoltion với 32 Kernel 5x5 -> Relu, tạo ra 32 feauture map
    layers.MaxPooling2D((2,2), padding='same'), # Max pooling giảm chiều 32 feature map của layer 1 (các feature xếp chồng lên nhau chứ ko tách ra)
    layers.Conv2D(64, (5,5), activation='relu', padding='same'), # Convolution bằng 64 kernel cỡ 5x5, ReLU -> 64 featur map mới
    layers.MaxPooling2D((2,2), padding='same'), # Max pooling các 64 feature map trong output của layer 2
    layers.Flatten(), # Chuyển các tensor 3D thành 1D để đưa vào lớp fully connected (vì FFN chỉ nhận input là vecto 1D)
    layers.Dense(512, activation='relu'), # Lớp Dense (1) có 512 neuron dày đặc, dùng hàm ReLU
    layers.Dense(2, activation='softmax') # Lớp Dense (2) có 2 node (để cho ra output phân loại), dùng Softmax đưa về xác suất thuộc lớp 0/1
])

# 3. Compile và huấn luyện

# Dùng thuật toán Adam để cập nhật trọng số; hàm 'categorical_crossentropy' dùng làm hàm mất mát; Chỉ số (accurancy) để đánh giá kết quả
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# epoch: số vòng huấn luyện,
# batch_size: số lượng mẫu được xử lý trong 1 lần cập nhật trọng số (1 batch) (ví dụ batch = 32 := mỗi lần update, mạng sẽ học 32 ảnh cùng 1 lúc)
# validation_split: tỉ lệ phần trăm của train sẽ tách ra để đánh giá tạm thời
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

# 4. Đánh giá
loss, acc = model.evaluate(x_test, y_test)
print(f"Test loss: {loss:.4f}, Test accuracy: {acc:.4f}")


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 48, 48, 32)     │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 24, 24, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 24, 24, 64)     │        51,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 12, 12, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 9216)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     4,719,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │         1,026 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,772,226 (18.20 MB)

 Trainable params: 4,772,226 (18.20 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 16s 296ms/step - accuracy: 0.8501 - loss: 0.3566 - val_accuracy: 1.0000 - val_loss: 4.5957e-04
Epoch 2/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 14s 288ms/step - accuracy: 1.0000 - loss: 3.8649e-04 - val_accuracy: 1.0000 - val_loss: 7.5224e-05
Epoch 3/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 20s 286ms/step - accuracy: 1.0000 - loss: 4.0770e-05 - val_accuracy: 1.0000 - val_loss: 4.1907e-05
Epoch 4/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 20s 273ms/step - accuracy: 1.0000 - loss: 3.9341e-05 - val_accuracy: 1.0000 - val_loss: 2.5910e-05
Epoch 5/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 21s 286ms/step - accuracy: 1.0000 - loss: 1.9873e-05 - val_accuracy: 1.0000 - val_loss: 1.9821e-05
Epoch 6/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 14s 292ms/step - accuracy: 1.0000 - loss: 2.4349e-05 - val_accuracy: 1.0000 - val_loss: 1.4658e-05
Epoch 7/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 20s 286ms/step - accuracy: 1.0000 - loss: 2.1396e-05 - val_accuracy: 1.0000 - val_loss: 1.1819e-05
Epoch 8/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 21s 292

In [22]:
import numpy as np
import pandas as pd

# Đọc file test
df_test = pd.read_csv("test.csv")  # Đổi lại đúng tên file nếu khác

# Xử lý dữ liệu pixels
pixels_test = np.array([list(map(int, row.split())) for row in df_test['pixels']])
pixels_test = pixels_test.reshape(-1, 48, 48, 1) / 255.0  # Chuẩn hóa [0,1] nếu lúc train cũng chuẩn hóa

# Dự đoán xác suất từng lớp
probs = model.predict(pixels_test)  # shape (300, 2)

# Lấy nhãn dự đoán (0 hoặc 1) theo xác suất lớn nhất
y_pred = np.argmax(probs, axis=1)

# Tạo DataFrame kết quả
df_result = pd.DataFrame({
    'predict': y_pred
})

# Lưu ra file CSV
df_result.to_csv('test_predict.csv', index=False)


208/208 ━━━━━━━━━━━━━━━━━━━━ 15s 72ms/step


In [21]:
'''
for i, row in enumerate(df_test['pixels']):
    if len(row.split()) != 2304:
        print(f"Row {i} has {len(row.split())} pixels")
'''

Row 374 has 67 pixels
